# **今季のお勧めアニメ紹介ツール**
アニメ情報サイトからあらすじを入手した上で、あらすじに含まれる単語とユーザーの指定したキーワードの類似性をword2vecを用いて計算し、最終的な類似度スコアの高い順にアニメを表示するツールを作成した。

In [ ]:
!pip install transformers fugashi unidic-lite ipadic gensim

import os
import random
import time
import urllib.request
import tarfile
import json

import requests
from bs4 import BeautifulSoup
import gensim
from sklearn.metrics.pairwise import cosine_similarity
from fugashi import Tagger

**各アニメのタイトルと、あらすじの書かれたページのURLの獲得**

In [ ]:
url = "https://anime.eiga.com/program/"  # アニメ情報サイト（アニメハック）のURL
res = requests.get(url)
res.encoding = res.apparent_encoding
soup = BeautifulSoup(res.text, "lxml")


# アニメのタイトルと詳細ページのURLを抽出し辞書を作成
anime_url = {}
for div1 in soup.find_all('div', attrs={"class": "animeSeasonItemWrapper"}):
    for div2 in div1.find_all('p', attrs={"class": "seasonAnimeTtl"}):
        next_page_url = div2.find('a')['href']
        #print(div2.text)
        #print(f"URL: {url[:-9]}{next_page_url}")
        anime_url[div2.text] = f"{url[:-9]}{next_page_url}"

**アニメのタイトルとそのあらすじを獲得**

In [ ]:
anime_abstract = {}
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.5 Safari/605.1.15"}

# それぞれのアニメのあらすじを抽出し辞書を作成
for title, url in anime_url.items():
    res = requests.get(url)
    res.encoding = res.apparent_encoding
    soup = BeautifulSoup(res.text, "lxml")
    dl = soup.find('dl', id='detailSynopsis')
    if dl:
        dd = dl.find('dd')
        if dd:
            story = dd.text.strip()
            anime_abstract[title] = story
    sleep_time = random.uniform(3.0, 5.0)
    time.sleep(sleep_time)

#print(anime_abstract)
#print(len(anime_abstract))
#with open("anime_abstract.json", "w", encoding="utf-8") as f:  # タイトルとあらすじの辞書を保存
    #json.dump(anime_abstract, f, ensure_ascii=False, indent=2)

**スクレイピングを省略する場合**

In [ ]:
"""
# スクレイピングに時間がかかるので、短縮する場合は添付したJSONファイルを適切な場ディレクトリに配置して読み込んでください。よろしくお願い致します。

with open("anime_abstract.json", "r", encoding="utf-8") as f:
    anime_abstract = json.load(f)
print(anime_abstract)
"""

'\n# スクレイピングに時間がかかるので、短縮する場合は添付したJSONファイルを適切な場ディレクトリに配置して読み込んでください。よろしくお願い致します。\n\nwith open("anime_abstract.json", "r", encoding="utf-8") as f:\n    anime_abstract = json.load(f)\nprint(anime_abstract)\n'

**fugashiを用いてあらすじから普通名詞を取り出す**

In [ ]:
anime_wakati = {}
tagger = Tagger()

for title, abstract in anime_abstract.items():
    wakati = []
    for word in tagger.parseToNodeList(abstract):
        if '普通名詞' in word.pos:
            wakati.append(word)
    anime_wakati[title] = wakati

#print(anime_wakati)

**word2vec学習済みモデルchiVeのデータを取得**

In [ ]:
url = "https://sudachi.s3-ap-northeast-1.amazonaws.com/chive/chive-1.2-mc90_gensim.tar.gz"
archive_path = "chive-1.2-mc90_gensim.tar.gz"

urllib.request.urlretrieve(url, archive_path)
with tarfile.open(archive_path, "r:gz") as tar:
    tar.extractall()

os.remove(archive_path)

chive = gensim.models.KeyedVectors.load('chive-1.2-mc90_gensim/chive-1.2-mc90.kv')  # version1.2, 最低頻度90のモデルを使用

**ユーザーのキーワードと各アニメのあらすじの類似度スコアを計算**

In [ ]:
# 以下のリストに,区切りで正規化表記のキーワードを入力
keyword_list = ['冒険', '宝', '犯罪']

similarity_score = {}
for title, tokens in anime_wakati.items():
    keyword_score = []
    for keyword in keyword_list:
        scores = []
        for token in tokens:
            try:
                # 各キーワードと、アニメのあらすじに含まれるそれぞれの普通名詞のコサイン類似度を計算
                score = cosine_similarity([chive[keyword]], [chive[token.surface]])[0][0]
                scores.append(score)
            except KeyError:  # 含まれない単語はパス
                continue
        keyword_score.append(scores)
    similarity_score[title] = keyword_score  # キーワードごとの類似度リストの入ったリストを、keyをタイトルとして辞書に格納

**最終類似度スコアの値に応じてユーザーへお勧めのアニメを紹介**

In [ ]:
# それぞれの（キーワードごとの）リストのトップ5の値を全て足して、最終的なアニメの類似度スコアとする

top_n = 5  # トップ何の値を足すか
final_score_dict = {}

for title, scores in similarity_score.items():
    total_score = 0
    for score_list in scores:
        top_scores = sorted(score_list, reverse=True)[:top_n]
        total_score += sum(top_scores)  # 各リストのTOP5の値を合計し最終スコアに
    final_score_dict[title] = total_score


# 類似度スコア順にお勧めのアニメを表示

num_recommendations = 5  # お勧めするアニメの数
recommend_anime = sorted(final_score_dict.items(), key=lambda x: x[1], reverse=True)[:num_recommendations]
print("あなたへのお勧めアニメ")
for i, (title, score) in enumerate(recommend_anime):
    print(f"お勧め度{i + 1}位 「{title}」 : {score:.4f}")

あなたへのお勧めアニメ
お勧め度1位 「勇者パーティーを追放された白魔導師、Sランク冒険者に拾われる ～この白魔導師が規格外すぎる～」 : 6.4290
お勧め度2位 「追放者食堂へようこそ！」 : 6.3244
お勧め度3位 「ポケットモンスター」 : 6.1138
お勧め度4位 「ONE PIECE（ワンピース）」 : 5.8887
お勧め度5位 「SAND LAND: THE SERIES」 : 5.7532
